**Date :** Created on Thursday January 7 2021

**Group 8 - Innovation**

**WMDSimilarityByFastText_v0** 

**@author :** Melina Audiger. 

**Description :** Word vectors (by FastText) are used  to calculate a vector for each article in order to calculate a WMD similarity between two articles. The save_art_similar function returns a Dataframe with the 10 most similar articles for each article with the corresponding score

**Note :** Our code works, however, it takes an enormous amount of time to perform calculations. Hence the fact that we did not launch the execution of the cells.

# Part 1 : Install / Download / Import Librairy

## Install librairy

In [ ]:
#!pip install gensim

## Import librairy

### - Usefull librairy :

In [30]:
import pandas as pd
from tqdm import trange
from google.colab import drive

### - Machine Learning librairy :

In [11]:
from gensim.models.keyedvectors import KeyedVectors

# Part 2 : Initialization of the model : KeyedVectors

In [17]:
# Connect the drive folder
drive.mount('/content/drive')

# My file path for the data
Helper_path : str = '/content/drive/MyDrive/data_interpromo/G8 Innovation/cc.fr.300.vec'

# My initialized model
Model = KeyedVectors.load_word2vec_format(Helper_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Part 3 : Data Loading

In [19]:
def Load_data(helper_path : str) -> pd.DataFrame:
    """Documentation
    
    Parameters :
        - helper_path : the file path

    Output (if exists) :
        - df : My Dataframe cleaned and reindexed

    """
    
    # Data Load with pandas librairy
    df = pd.read_csv(Helper_path)

    # Drop articles with no content
    df = df[df['art_content'] != '']

    # Reset my dataframe index
    df = df.reset_index(drop = True)
    
    # Returns my clean dataframe
    return df

In [23]:
# My file path for the fonction
Helper_path : str = '/content/drive/MyDrive/data_interpromo/Data/abstract_v1.csv'

# My DataFrame variable
My_data : pd.DataFrame = Load_data(Helper_path)

# To show my DataFrame
My_data.head(10)

,art_id,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag,art_clean,abstract_sentence,abstract_words
0,1,le FNCDG et l’ andcdg avoir publier en septemb...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,fr,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/09/im...,NaN,NaN,fncdg andcdg avoir publier septembre 9em editi...,fncdg andcdg avoir publier septembre 9em editi...,fncdg andcdg avoir publier septembre 9em editi...
1,2,malgré le levée un mesure de confinement le 11...,"<p style=""text-align: justify;"">Malgré la levé...",17 mars 2020,fr,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/03/co...,NaN,NaN,malgre levee mesure confinement 11 mai 2020 pl...,malgre levee mesure confinement 11 mai 2020 pl...,malgre levee mesure confinement 11 mai 2020 pl...
2,25,quel être le objectif poursuivre par le gouver...,"<p style=""text-align: justify;""><strong>Quels ...",24 octobre 2019,fr,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2019/10/in...,NaN,NaN,quel etre objectif poursuivre gouvernement cad...,quel etre objectif poursuivre gouvernement cad...,quel etre objectif poursuivre gouvernement cad...
3,27,"le journée thématique , qui avoir lieu durant ...","<p style=""text-align: justify;""><strong>La jo...",31 mai 2017,fr,Journée Thématique FNCDG « Les services de san...,http://fncdg.com/journee-thematique-fncdg-les-...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/05/pu...,NaN,NaN,journee thematique avoir lieu durant salon pre...,journee thematique avoir lieu durant salon pre...,journee thematique avoir lieu durant salon pre...
4,28,le 1ère journée thématique en région sur le th...,"<p style=""text-align: justify;"">La 1<sup>ère</...",13 mars 2017,fr,Journée Thématique FNCDG « Vers de nouveaux mo...,http://fncdg.com/journee-thematique-fncdg-vers...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/03/Sa...,NaN,NaN,1ere journee thematique region theme vers nouv...,1ere journee thematique region theme vers nouv...,1ere journee thematique region theme vers nouv...
5,30,l’ un un innovation de le loi n degré 2019 - 8...,"<p style=""text-align: justify;"">L’une des inno...",22 octobre 2020,fr,La publication d’un guide d’accompagnement à l...,http://fncdg.com/la-publication-dun-guide-dacc...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/10/LG...,NaN,NaN,innovation loi degre 2019 828 6 aout 2019 dire...,innovation loi degre 2019 828 6 aout 2019 dire...,innovation loi degre 2019 828 6 aout 2019 dire...
6,31,"le FNCDG mener , en collaboration avec d’ autr...","<p style=""text-align: justify;"">La FNCDG mène,...",10 décembre 2020,fr,La publication d’un guide de sensibilisation a...,http://fncdg.com/la-publication-dun-guide-de-s...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/12/im...,NaN,NaN,fncdg mener collaboration autre partenaire cam...,fncdg mener collaboration autre partenaire cam...,fncdg mener collaboration autre partenaire cam...
7,32,"créer pour et par le décideur territorial , ét...","<p style=""text-align: justify;"">Créé pour et p...",24 février 2017,fr,Lancement du réseau Étoile,http://fncdg.com/lancement-du-reseau-etoile/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/02/re...,NaN,NaN,creer decideur territorial etoile etre tout pr...,creer decideur territorial etoile etre tout pr...,creer decideur territorial etoile etre tout pr...
8,34,le décret n

# Part 4 : Detection of similar articles

In [37]:
def Save_articles_similar(df_id, df_art_clean, nb_similar: int) -> pd.DataFrame:
    """Documentation
  
    Parameters :
      - df_id : identification of article
      - df_art_clean : coordinates of article
      - nb_similar : number of article similar to save
  
    Output (if exists) :
      - df_art_similar : My list of similars article for everyone
  
    """

    # My personnal list
    df_columns : list = ['art_id','art_sim','score']

    # My initialized DataFrame
    df_art_similar : pd.DataFrame = pd.DataFrame(columns = df_columns)

    # Step 1 : Browse all the article
    for i in trange(len(df_id) , position = 0 , leave = True):
    
        # My initialized list
        list_similar : list = []

        # Step 2 : Browse other article for an article
        for j in range(len(df_id)):
      
            # Check difference 
            if i != j:

                # My first item 
                item_1 : list = df_art_clean[i].lower().split()

                # My second item
                item_2 : list = df_art_clean[j].lower().split()

                # Distance calcul 
                distance_items : float = Model.wmdistance(item_1, item_2)

                # Retrieving the identifier
                df_id_j : int = df_id[j]

                # Save the similarity
                list_similar.append([df_id_j, distance_items])

        # Check if too many similar items
        if len(list_similar) > nb_similar:

            # Collapse the list
            list_similar = sorted(list_similar, \
                            key = operator.itemgetter(1))[0:nb_similar]

        # Step 3 : Save similar articles and score 
        for listing in list_similar:

            # Retrieving the identifier
            df_id_i : int = df_id[i]

            # Retrieving similarities list
            sim_list : int = listing[0]
      
            # Retrieving score
            sim_score : float = listing[1]
      
            # List creation
            df_art_similar = df_art_similar.append({'art_id' : df_id_i, \
                                                    'art_sim' : sim_list, \
                                                    'score' : sim_score}, \
                                                    ignore_index = True)

    # Delete infinity values
    df_art_similar = df_art_similar.drop(df_art_similar[df_art_similar['score'] == np.inf].index)

    # Change of type 
    df_art_similar['art_id'] = df_art_similar['art_id'].astype('long')
    df_art_similar['art_sim'] = df_art_similar['art_sim'].astype('long')

    # Return my similarity DataFrame
    return df_art_similar

In [ ]:
# Fonction Application
Df_art_similar = Save_articles_similar(My_data['art_id'], My_data['art_clean'], 10)

# Part 5 : Export data

In [ ]:
# The .csv export data
Df_art_similar.to_csv('article_similar.csv', index = False, encoding = 'utf-8')